In [ ]:
!pip install dpkt
!pip install pygeoip

import dpkt
import socket
import pygeoip

gi = pygeoip.GeoIP('GeoLiteCity.dat')

#function to translate IP addresses into geolocations
#src variable is source IP addr
def retKML(dstip, srcip):
  dst = gi.record_by_name(dstip)
  src = gi.record_by_name('xx.xxx.xxx.xxx')
  try:
    dstlongitude = dst['longitude']
    dstlatitude = dst['latitude']
    srclongitude = src['longitude']
    srclatitude = src['latitude']
    kml = (
        '<Placemark>\n'
        '<name>%s</name>\n'
        '<extrude>1</extrude>\n'
        '<tessellate>1</tessellate>\n'
        '<styleUrl>#transBluePoly</styleUrl>\n'
        '<LineString>\n'
        '<coordinates>%6f,%6f\n%6f,%6f</coordinates>\n'
        '<LineString>\n'
        '<Placemark>\n'
    )%(dstip, dstlongitude, dstlatitude, srclongitude, srclatitude)
    return kml
  except:
    return ''

#method to create the data by looping through each packet to fetch src and dest IP addr
def plotIPs(pcap):
  kmlPts = ''
  for (ts, buf) in pcap:
    try:
      eth = dpkt.ethernet.Ethernet(buf)
      ip = eth.data
      src = socket.inet_ntoa(ip.src)
      dst = socket.inet_ntoa(ip.dst)
      KML = retKML(dst, src)
      kmlPts = kmlPts + KML
    except:
      pass
  return kmlPts

#denoting xml file and creating line styling for the map traces
def main():
  f = open('8.21.23.855AM.pcap', 'rb')
  pcap = dpkt.pcap.Reader(f)
  kmlheader = '<?xml version="1.0" encoding="UTF-8"?> \n<kml xmlns="http://www.opengis.net/kml/2.2">\n<Document>\n'\
  '<Style id="transBluePoly">' \
        '<LineStyle>' \
        '<width>1.5</width>' \
        '<color>501400E6</color>' \
        '</LineStyle>' \
        '</Style>'
  kmlfooter = '</Document>\n</kml>\n'
  kmldoc=kmlheader + plotIPs(pcap) + kmlfooter
  print(kmldoc)

if __name__ == '__main__':
  main()

